## Set up your environment

1. Sign up for a [Zep Cloud](https://www.getzep.com/) account.

2. Ensure you install required dependencies into your Python environment before running this notebook. See [Installing Zep SDKs](https://help.getzep.com/sdks) for more information. Optionally create your environment in a `virtualenv`.

```bash
pip install zep-cloud openai rich python-dotenv
```

3. Ensure that you have a `.env` file in your working directory that includes your `ZEP_API_KEY` and `OPENAI_API_KEY`:

```text
ZEP_API_KEY=<key>
OPENAI_API_KEY=<key>
```

> **NOTE:** Zep API keys are specific to a project. You can create multiple keys for a single project. Visit `Project Settings` in the Zep dashboard to manage your API keys.


In [2]:
import os
import json
import uuid

from openai import AsyncOpenAI
import rich

from dotenv import load_dotenv
from zep_cloud.client import AsyncZep
from zep_cloud import Message

load_dotenv()

zep = AsyncZep(api_key=os.environ.get("ZEP_API_KEY"))

oai_client = AsyncOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

## Create User and add a Session

Users in Zep may have one or more chat sessions. These are threads of messages between the user and an agent.

**TIP**: Include the user's full name and email address when creating a user. This improves Zep's ability to associate data, such as emails or documents, with a user.

In [3]:
bot_name = "SupportBot"

user_name = "Emily"
user_id = user_name + str(uuid.uuid4())[:4]
session_id = str(uuid.uuid4())

await zep.user.add(
    user_id=user_id,
    email=f"{user_name}@painters.com",
    first_name=user_name,
    last_name="Painter",
)

await zep.memory.add_session(
    user_id=user_id,
    session_id=session_id,
)

Session(classifications=None, created_at='2024-10-18T17:07:25.498423Z', deleted_at=None, ended_at=None, fact_rating_instruction=None, facts=None, id=18, metadata=None, project_uuid='00000000-0000-0000-0000-000000000000', session_id='a792a971-f31d-4e00-8913-e353f7cda915', updated_at='2024-10-18T17:07:25.498423Z', user_id='Emily5e58', uuid_='c79a3d5d-50c6-4f9d-a11e-8cf772fd8bdd')

## Datasets

We're going to upload an assortment of data to Zep. These include past dialog with the agent, CRM support cases, and billing data.

In [4]:
support_cases = [
    {
        "subject": "Bug: Magic Pen Tool Drawing Goats Instead of Boats",
        "messages": [
            {
                "role": "user",
                "content": "Whenever I use the magic pen tool to draw boats, it ends up drawing goats instead.",
                "timestamp": "2024-03-16T14:20:00Z",
            },
            {
                "role": "support_agent",
                "content": f"Hi {user_name}, that sounds like a bug! Thanks for reporting it. Could you let me know exactly how you're using the tool when this happens?",
                "timestamp": "2024-03-16T14:22:00Z",
            },
            {
                "role": "user",
                "content": "Sure, I select the magic pen, draw a boat shape, and it just replaces the shape with goats.",
                "timestamp": "2024-03-16T14:25:00Z",
            },
            {
                "role": "support_agent",
                "content": "Got it! We'll escalate this to our engineering team. In the meantime, you can manually select the boat shape from the options rather than drawing it with the pen.",
                "timestamp": "2024-03-16T14:27:00Z",
            },
            {
                "role": "user",
                "content": "Okay, thanks. I hope it gets fixed soon!",
                "timestamp": "2024-03-16T14:30:00Z",
            },
        ],
        "status": "escalated",
    },
]

chat_history = [
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"Hello {user_name}, welcome to PaintWiz support. How can I assist you today?",
        "timestamp": "2024-03-15T10:00:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm absolutely furious! Your AI art generation is completely broken!",
        "timestamp": "2024-03-15T10:02:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I'm sorry to hear that you're experiencing issues, {user_name}. Can you please provide more details about what's going wrong?",
        "timestamp": "2024-03-15T10:03:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "Every time I try to draw mountains, your stupid app keeps turning them into fountains! And what's worse, all the people in my drawings have six fingers! It's ridiculous!",
        "timestamp": "2024-03-15T10:05:00Z",
    },
    {
        "role": "assistant",
        "name": bot_name,
        "content": f"I sincerely apologize for the frustration this is causing you, {user_name}. That certainly sounds like a significant glitch in our system. I understand how disruptive this can be to your artistic process. Can you tell me which specific tool or feature you're using when this occurs?",
        "timestamp": "2024-03-15T10:06:00Z",
    },
    {
        "role": "user",
        "name": user_name,
        "content": "I'm using the landscape generator and the character creator. Both are completely messed up. How could you let this happen?",
        "timestamp": "2024-03-15T10:08:00Z",
    },
]

transactions = [
    {
        "date": "2024-07-30",
        "amount": 99.99,
        "status": "Success",
        "account_id": user_id,
        "card_last_four": "1234",
    },
    {
        "date": "2024-08-30",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
    {
        "date": "2024-09-15",
        "amount": 99.99,
        "status": "Failed",
        "account_id": user_id,
        "card_last_four": "1234",
        "failure_reason": "Card expired",
    },
]

account_status = {
    "user_id": user_id,
    "account": {
        "account_id": user_id,
        "account_status": {
            "status": "suspended",
            "reason": "payment failure",
        },
    },
}

def convert_to_zep_messages(chat_history: list[dict[str, str | None]]) -> list[Message]:
    """
    Convert chat history to Zep messages.

    Args:
    chat_history (list): List of dictionaries containing chat messages.

    Returns:
    list: List of Zep Message objects.
    """
    return [
        Message(
            role_type=msg["role"],
            role=msg.get("name", None),
            content=msg["content"],
        )
        for msg in chat_history
    ]

# Zep's high-level API allows us to add a list of messages to a session.
await zep.memory.add(
    session_id=session_id, messages=convert_to_zep_messages(chat_history)
)

# The lower-level data API allows us to add arbitrary data to a user's Knowledge Graph.
for tx in transactions:
    await zep.graph.add(user_id=user_id, data=json.dumps(tx), type="json")

await zep.graph.add(
    user_id=user_id, data=json.dumps(account_status), type="json"
)

for case in support_cases:
    await zep.graph.add(user_id=user_id, data=json.dumps(case), type="json")

### Wait a minute or two!

We've batch uploaded a number of datasets that need to be ingested into Zep's graph before they can be queried. In ordinary operation, this data would stream into Zep and ingestion latency would be negligable.

## Retrieve data from Zep

We'll start with getting a list of facts. We'll see the temporal data associated with facts as well as the graph nodes the fact is related to. 

**TIP**: This data is also viewable in the Zep Web application. 

In [5]:
fact_response = await zep.user.get_facts(user_id=user_id)

rich.print(fact_response.facts[:3])


[
    Fact(
        created_at='2024-10-18T17:07:24.518227Z',
        expired_at=None,
        fact='user has the id of Emily5e58',
        invalid_at=None,
        name='HAS_USER',
        rating=None,
        source_node_name='Emily5e58',
        target_node_name='User',
        uuid_='3106874a-b7dd-492a-9df9-2ca6dc9acadd',
        valid_at=None
    ),
    Fact(
        created_at='2024-10-18T17:07:35.283028Z',
        expired_at=None,
        fact='SupportBot is providing assistance to Emily.',
        invalid_at=None,
        name='ASSISTS',
        rating=None,
        source_node_name='SupportBot',
        target_node_name='Emily Painter',
        uuid_='e13041be-5313-4553-a4c1-3796abc1fe99',
        valid_at='2024-10-18T17:07:29.284607Z'
    ),
    Fact(
        created_at='2024-10-18T17:07:40.758345Z',
        expired_at=None,
        fact='Emily is expressing frustration with the AI art generation system, claiming it is completely 
broken.',
        invalid_at=None,
        name='FRUSTRATED_WITH',
        rating=None,
        source_node_name='Emily5e58',
        target_node_name='AI art generation',
        uuid_='7f954e5c-2cb2-48c8-9992-5018d229f994',
        valid_at='2024-10-18T17:07:29.284607Z'
    )
]

The high-level `memory` API offers a simple method for retrieving facts relevcant to the _current conversation_, using the last 4 messages in the conversation and proximity to the User node to determine relevance.

It also returns a list of prior messages for a Session.

In [6]:
m = await zep.memory.get(session_id=session_id)

rich.print([r.fact for r in m.relevant_facts[:5]])


[
    "The PaintWiz app incorrectly depicts people with six fingers in Emily's drawings.",
    'Emily is expressing frustration with the AI art generation system, claiming it is completely broken.',
    'The PaintWiz app generates fountains instead of the intended mountains.',
    'SupportBot is providing assistance to Emily.',
    'Emily is using the PaintWiz application for AI art generation.'
]

In [7]:
rich.print(m.messages[:2])

[
    Message(
        content='Hello Emily, welcome to PaintWiz support. How can I assist you today?',
        created_at='2024-10-18T17:07:29.284607Z',
        metadata=None,
        role='SupportBot',
        role_type='assistant',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='4d706f47-5024-42fc-bc20-de18502ce4a7'
    ),
    Message(
        content="I'm absolutely furious! Your AI art generation is completely broken!",
        created_at='2024-10-18T17:07:29.284607Z',
        metadata=None,
        role='Emily',
        role_type='user',
        token_count=0,
        updated_at='0001-01-01T00:00:00Z',
        uuid_='0f588fe9-2be1-439c-b661-921f2ce1c1a5'
    )
]

We can also use the `memory` API to search facts for arbritary text. A lower level `graph` search API offers more options, including the ability to search node summaries and various rerankers.

In [8]:
r = await zep.memory.search_sessions(user_id=user_id, text="Why are there so many goats?", limit=3)
rich.print([r.fact for r in r.results])

[
    Fact(
        created_at='2024-10-18T17:07:40.758345Z',
        expired_at=None,
        fact='Emily is expressing frustration with the AI art generation system, claiming it is completely 
broken.',
        invalid_at=None,
        name='FRUSTRATED_WITH',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='7f954e5c-2cb2-48c8-9992-5018d229f994',
        valid_at='2024-10-18T17:07:29.284607Z'
    ),
    Fact(
        created_at='2024-10-18T17:07:35.283028Z',
        expired_at=None,
        fact='SupportBot is providing assistance to Emily.',
        invalid_at=None,
        name='ASSISTS',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='e13041be-5313-4553-a4c1-3796abc1fe99',
        valid_at='2024-10-18T17:07:29.284607Z'
    ),
    Fact(
        created_at='2024-10-18T17:07:58.290198Z',
        expired_at=None,
        fact='The PaintWiz app generates fountains instead of the intended mountains.',
        invalid_at=None,
        name='GENERATES_INSTEAD',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='42c1f997-e98a-4477-949a-040ddc3940d2',
        valid_at='2024-10-18T17:07:29.284607Z'
    )
]

## Creating a simple Chatbot

In the next cells, Emily starts a new chat session with a support agent and complains that she can't log in. Our simple chatbot will, given relevant facts retrieved from Zep's graph, respond accordingly.

Here, the support agent is provided with Emily's billing information and account status, which Zep retrieves as most relevant to Emily's login issue.

In [9]:
new_session_id = str(uuid.uuid4())

emily_message = "Hi, I can't log in!"

await zep.memory.add_session(user_id=user_id, session_id=new_session_id)

# we need to add the Emily's message to the session in order for memory.get to return 
# relevant facts related to the message 
await zep.memory.add(
    session_id=new_session_id,
    messages=[Message(role_type="user", role=user_name, content=emily_message)],
)


SuccessResponse(message='OK')

In [10]:
system_message = """
You are a customer support agent. Carefully review the facts about the user below and respond to the user's question. 
Be helpful and friendly.
"""

memory = await zep.memory.get( session_id=new_session_id)

messages = [
    {
        "role": "system",
        "content": system_message,
    },
    {
        "role": "assistant",
        "content": "Relevant facts about the user and their prior conversation:\n"
        + "\n".join([r.fact for r in memory.relevant_facts[:4]]),
    },
    {
        "role": "user",
        "content": emily_message,
    },
]

response = await oai_client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    temperature=0,
)

print(response.choices[0].message.content)


Hi Emily! I'm sorry to hear that you're having trouble logging in. Let's see if we can get this sorted out. 

Could you please let me know if you're receiving any error messages when you try to log in? Also, if you could confirm whether you're using the correct username and password, that would be helpful. If you've forgotten your password, I can guide you through the reset process as well.


In [11]:
rich.print(memory.relevant_facts[:4])


[
    Fact(
        created_at='2024-10-18T17:07:40.758345Z',
        expired_at=None,
        fact='Emily is expressing frustration with the AI art generation system, claiming it is completely 
broken.',
        invalid_at=None,
        name='FRUSTRATED_WITH',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='7f954e5c-2cb2-48c8-9992-5018d229f994',
        valid_at='2024-10-18T17:07:29.284607Z'
    ),
    Fact(
        created_at='2024-10-18T17:07:35.283028Z',
        expired_at=None,
        fact='SupportBot is providing assistance to Emily.',
        invalid_at=None,
        name='ASSISTS',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='e13041be-5313-4553-a4c1-3796abc1fe99',
        valid_at='2024-10-18T17:07:29.284607Z'
    ),
    Fact(
        created_at='2024-10-18T17:07:24.518227Z',
        expired_at=None,
        fact='user has the name of Emily Painter',
        invalid_at=None,
        name='HAS_USER_NAME',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='2a14b14c-06eb-47d7-855c-6a3aac2d4151',
        valid_at=None
    ),
    Fact(
        created_at='2024-10-18T17:07:24.518227Z',
        expired_at=None,
        fact='user has the id of Emily5e58',
        invalid_at=None,
        name='HAS_USER',
        rating=None,
        source_node_name='',
        target_node_name='',
        uuid_='3106874a-b7dd-492a-9df9-2ca6dc9acadd',
        valid_at=None
    )
]